In [1]:
import pyomo.environ as pyo
import idaes
from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock)
import idaes.core.util.scaling as iscale
from idaes.models_extra.power_generation.properties.natural_gas_PR import get_prop
from idaes.models_extra.power_generation.unit_models.soec_design import EosType
import pytest



In [17]:
import importlib
import SOFC 
importlib.reload(SOFC)
from SOFC import SofcDesign


def sofc_example_flowsheet(eos=EosType.PR):
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    air_cathode = {
        "O2":0.2074,
        "H2O":0.0099,
        "CO2":0.0003,
        "N2":0.7732,
        "Ar":0.0092,
    }

    fuel_anode = {
        "H2": 0.99,
        "H2O": 0.01,
    }

    m.fs.air_side_property_package = GenericParameterBlock(
        **get_prop(air_cathode, {"Vap"}, eos=eos),
        doc="Air property parameters",
    )
    m.fs.fuel_side_property_package = GenericParameterBlock(
        **get_prop(fuel_anode, {"Vap"}, eos=eos),
        doc="Flue gas property parameters",
    )
    m.fs.sofc = SofcDesign(
        air_side_property_package=m.fs.air_side_property_package,
        fuel_side_property_package=m.fs.fuel_side_property_package,
        reaction_eos=eos
    )

    m.fs.sofc.fuel_side_inlet.temperature.fix(1023)
    m.fs.sofc.fuel_side_inlet.pressure.fix(20e5)
    m.fs.sofc.fuel_side_inlet.flow_mol.fix(2)

    for (t, i), c in m.fs.sofc.fuel_side_inlet.mole_frac_comp.items():
        c.fix(fuel_anode[i])

    m.fs.sofc.air_side_inlet.temperature.fix(1023)
    m.fs.sofc.air_side_inlet.pressure.fix(20e5)
    m.fs.sofc.air_side_inlet.flow_mol.fix(2)

    for (t, i), c in m.fs.sofc.air_side_inlet.mole_frac_comp.items():
        c.fix(air_cathode[i])
    # m.fs.sofc.hydrogen_side_outlet_temperature.fix(1023)
    # m.fs.sofc.oxygen_side_outlet_temperature.fix(1023)
    
    m.fs.sofc.fuel_util.fix(0.7)
    m.fs.sofc.number_of_cells.fix(80)
    m.fs.sofc.cell_area.fix(320 * 1e-4)
    #iscale.calculate_scaling_factors(m)
    m.fs.sofc._scaling_guess()
    m.fs.sofc.initialize(optarg={"max_iter":100, "tol":1e-5})
    return m

# create a flowsheet with the SOEC model
model = sofc_example_flowsheet()

[DEBUG] Inlet flow of O2: 1600.0
[DEBUG] Inlet flow of H2O: 1600.0
[DEBUG] Inlet flow of CO2: 1600.0
[DEBUG] Inlet flow of N2: 1600.0
[DEBUG] Inlet flow of Ar: 1600.0
{'H2', 'O2', 'H2O'}
{'H2O', 'Ar', 'N2', 'CO2', 'O2'}
2025-05-27 12:19:45 [INFO] idaes.init.fs.sofc.fuel_in_trans.properties_in: Starting initialization
2025-05-27 12:19:45 [INFO] idaes.init.fs.sofc.fuel_in_trans.properties_in: Property initialization: optimal - Optimal Solution Found.
2025-05-27 12:19:45 [INFO] idaes.init.fs.sofc.fuel_in_trans.properties_out: Starting initialization
2025-05-27 12:19:45 [INFO] idaes.init.fs.sofc.fuel_in_trans.properties_out: Property initialization: optimal - Optimal Solution Found.
2025-05-27 12:19:45 [INFO] idaes.init.fs.sofc.fuel_in_trans.properties_out: Property package initialization: optimal - Optimal Solution Found.
2025-05-27 12:19:45 [INFO] idaes.init.fs.sofc.fuel_in_trans: Initialization Complete optimal - Optimal Solution Found.
2025-05-27 12:19:45 [INFO] idaes.init.fs.sofc.air_

InitializationError: SOFC failed to initialize.